In [1]:
!pip install -q accelerate==0.21.0 bitsandbytes==0.40.2 transformers==4.31.0 datasets evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/116.9 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import Dataset
import torch
import random
import tempfile
import logging
import random
import numpy as np
import os
import yaml
import time

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate
import nltk
nltk.download("punkt", quiet=True)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_fscore_support

logger = logging.getLogger(__name__)
global_config = None

In [4]:
# checking avalability of cuda in device
device_count = torch.cuda.device_count()
if device_count > 0:
    logger.debug("Select GPU device")
    device = torch.device("cuda")
else:
    logger.debug("Select CPU device")
    device = torch.device("cpu")

In [6]:
# Reading the training, validation, master category data
master_Cat_df = pd.read_csv("/content/master_category_data.csv")
print(master_Cat_df.count())

train_df = pd.read_csv("/content/train_data.csv")
print(train_df.count())


val_df = pd.read_csv("/content/validation_data.csv")
print(val_df.count())

## Reading predicted testing data having predCat1
test_df = pd.read_csv("/content/testing_data_pred_Cat_1_2.csv")
print(test_df.count())

Unnamed: 0    9995
Cat1          9995
Cat2          9995
Cat3          9995
dtype: int64
Unnamed: 0       7496
productId        7496
Title            7496
userId           7496
Time             7496
Text             7496
Cat1             7496
Cat2             7496
Cat3             7496
clean_Title      7496
clean_Text       7496
combined_text    7496
dtype: int64
Unnamed: 0       1499
productId        1499
Title            1499
userId           1499
Time             1499
Text             1499
Cat1             1499
Cat2             1499
Cat3             1499
clean_Title      1499
clean_Text       1499
combined_text    1499
dtype: int64
Unnamed: 0.2     1000
Unnamed: 0.1     1000
Unnamed: 0       1000
productId        1000
Title            1000
userId           1000
Time             1000
Text             1000
Cat1             1000
Cat2             1000
Cat3             1000
clean_Title      1000
clean_Text       1000
combined_text    1000
pred_Cat1        1000
pred_Cat2        1000
dtype

In [7]:
#replacing spaces between words of Cat3 with "_"
#As in few records, suppose Cat3 has "foods", and in another case it has Cat3 "baby foods", 
#then tokeinzer is treating 2nd case as nested labels- treating "foods" in "baby foods" as different labels

master_Cat_df['Cat3']=master_Cat_df['Cat3'].apply(lambda x : x.replace(" ","_"))
train_df['Cat3']=train_df['Cat3'].apply(lambda x : x.replace(" ","_"))
val_df['Cat3']=val_df['Cat3'].apply(lambda x : x.replace(" ","_"))
# test_df['pred_Cat3']=test_df['pred_Cat3'].apply(lambda x : x.replace(" ","_"))

In [8]:
# Function to create the QA prompt for each row (without revealing the correct Cat3)
def create_prompt_for_Cat3(row, category1_column, category2_column, df):
    
    # Extract context and category
    clean_text = row['combined_text']
    Cat1 = row[category1_column]
    Cat2 = row[category2_column]

    # Filter the DataFrame to only include rows where Cat1 and Cat2 match the current row's Cat1 and Cat2
    filtered_df = df[(df['Cat1'] == Cat1) & (df['Cat2'] == Cat2)]

    # Get unique Cat3 options from the filtered DataFrame
    unique_Cat3_options = filtered_df['Cat3'].unique().tolist()

    # Shuffle the options
    random.shuffle(unique_Cat3_options)

    # Format the options into A/B/C/D... format
    options = {chr(65 + i): option for i, option in enumerate(unique_Cat3_options)}

    # Create the final prompt without the answer
    prompt = (
        f"Context: {clean_text}\n\n"
        f"Hint: This is a {Cat1} product in the {Cat2} sub-category.\n\n"
        f"Question: What is the most appropriate specific category for this product?\n\n"
        f"Options:\n"
    )

    # Add the options list to the prompt
    options_text = "".join([f"{letter}) {option}\n" for letter, option in options.items()])

    # Concatenate the final prompt and return it
    return prompt + options_text



In [9]:
# Generate prompts for each row (for training/validation/testing)
train_df['question'] = train_df.apply(lambda row: create_prompt_for_Cat3(row, "Cat1", "Cat2" ,master_Cat_df), axis=1)
val_df['question'] = val_df.apply(lambda row: create_prompt_for_Cat3(row, "Cat1", "Cat2" ,master_Cat_df), axis=1)

#Used predCat1 (from Level 1 Classifier) and predCat2(from Level2 classifier ) to perform hierarchical classifcation
test_df['question'] = test_df.apply(lambda row: create_prompt_for_Cat3(row, "pred_Cat1","pred_Cat2", master_Cat_df), axis=1)

In [10]:
train_df['question'][0]

'Context: the name of the product is remington nedhnumber titanium personal trimmer for men and women nose  ear trimmer the review of the product given by the user after purchasing it is i purchased my remington nedhnumber titanium personal trimmer at walgreens and have used it for almost two years ive had no problem with it trimming nose hair sufficiently but it is a little difficult to cut those just inside the base of the nose without doing some simple contortionist feats this is the only trimmer of this sort ive ever used i only use it for trimming nose hair so i cant compare it against other brands or rate it for any uses other than that purpose ive never been nicked while using the trimmer and it works well with fresh or recharged batteries it uses a single aa cell battery the trimmer has a single switch that slides onoff with only one speed its easy to clean just remove and rinse off the cutter head under hot running water and youre done the package includes the trimmer ear hair

### Example prompt

Context: the name of the product is remington nedhnumber titanium personal trimmer for men and women nose  ear trimmer the review of the product given by the user after purchasing it is i purchased my remington nedhnumber titanium personal trimmer at walgreens and have used it for almost two years ive had no problem with it trimming nose hair sufficiently but it is a little difficult to cut those just inside the base of the nose without doing some simple contortionist feats this is the only trimmer of this sort ive ever used i only use it for trimming nose hair so i cant compare it against other brands or rate it for any uses other than that purpose ive never been nicked while using the trimmer and it works well with fresh or recharged batteries it uses a single aa cell battery the trimmer has a single switch that slides onoff with only one speed its easy to clean just remove and rinse off the cutter head under hot running water and youre done the package includes the trimmer ear hair cutter head and a clear cutter cap this is a nice and inexpensive trimmerplease rate this review if you found it helpful

Hint: This is a health personal care product in the personal care sub-category.

Question: What is the most appropriate specific category for this product?

Options:
A) body_art
B) foot_care
C) lip_care_products
D) feminine_care
E) ear_care
F) deodorants_antiperspirants
G) eye_care
H) oral_hygiene
I) shaving_hair_removal
J) unknown


In [11]:
train_df['labels'] = train_df['Cat3']
val_df['labels'] = val_df['Cat3']

# Convert DataFrame to HuggingFace Dataset format
train_dataset = Dataset.from_pandas(train_df[['question', 'labels']])
val_dataset = Dataset.from_pandas(val_df[['question', 'labels']])


In [12]:
# Load pre-trained T5 model and tokenizer
model_name = "google/flan-t5-base"  # You can use "t5-base" or "t5-large" for bigger models
tokenizer = T5Tokenizer.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token
model = T5ForConditionalGeneration.from_pretrained(model_name,torch_dtype=torch.bfloat16)
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [13]:
# Tokenize inputs and outputs
def preprocess_data(examples):
    model_inputs = tokenizer(examples['question'], max_length=512, truncation=True, padding=True)

    # Tokenize the target labels (categories)
    labels = tokenizer(examples['labels'], max_length=10, truncation=True, padding=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing
train_dataset = train_dataset.map(preprocess_data, batched=True)
val_dataset = val_dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/7496 [00:00<?, ? examples/s]

Map:   0%|          | 0/1499 [00:00<?, ? examples/s]

In [14]:
print(train_dataset)
print(train_dataset.shape)
print(val_dataset.shape)

Dataset({
    features: ['question', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 7496
})
(7496, 4)
(1499, 4)


# Fine Tuning Seq2Seq Flan T5 model

In [15]:
# from peft import LoraConfig, get_peft_model, TaskType

# # LoRA Configuration
# lora_config = LoraConfig(
#     r=16,  # Rank,
#     lora_alpha=32,  # Alpha parameter
#     lora_dropout=0.1,  # Dropout
#     task_type=TaskType.SEQ_2_SEQ_LM  # Task type for T5
# )

# # Prepare the model with LoRA
# model = get_peft_model(model, lora_config)

### Defining Compute metrics
- Used Rogue-score as compute metrics because there are multiples tokens (2-3) in output sequence in few cases, rogue would help to check the overlap among the tokens of ground truth and predict sequence. 

In [16]:
metric = evaluate.load("rouge")

In [17]:
def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

   return result

In [18]:
#Defining model parameters
L_RATE = 3e-4
BATCH_SIZE = 16
PER_DEVICE_EVAL_BATCH = 8
WEIGHT_DECAY = 0.01
NUM_EPOCHS=3

In [19]:
training_args = Seq2SeqTrainingArguments(
   output_dir="./product_Cat3_classification",
   evaluation_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True
)

In [20]:
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=train_dataset,
   eval_dataset=val_dataset,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics
)

### Training and evaluation on validation set

In [21]:
# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,0.121744,0.782284,0.508117,0.781873,0.781743
2,0.818000,0.100112,0.813491,0.534134,0.813655,0.812765
3,0.116600,0.100169,0.820572,0.539137,0.820066,0.819432


TrainOutput(global_step=1407, training_loss=0.3601663314287935, metrics={'train_runtime': 4175.9053, 'train_samples_per_second': 5.385, 'train_steps_per_second': 0.337, 'total_flos': 1.5398823341850624e+16, 'train_loss': 0.3601663314287935, 'epoch': 3.0})

In [22]:
trainer.evaluate()

{'eval_loss': 0.1001688688993454,
 'eval_rouge1': 0.8205716509418979,
 'eval_rouge2': 0.5391372025794974,
 'eval_rougeL': 0.8200657581244639,
 'eval_rougeLsum': 0.819432002287239,
 'eval_runtime': 212.543,
 'eval_samples_per_second': 7.053,
 'eval_steps_per_second': 0.885,
 'epoch': 3.0}

In [23]:
# saving the fine tuned model
trainer.save_model('product_cat3_classifier')
tokenizer.save_pretrained('product_cat3_classifier')


('product_cat3_classifier/tokenizer_config.json',
 'product_cat3_classifier/special_tokens_map.json',
 'product_cat3_classifier/spiece.model',
 'product_cat3_classifier/added_tokens.json')

### Defined a function to generate inference sing fine tuned model

In [24]:
def inference(text, model, tokenizer, max_input_tokens=512, max_output_tokens=10):
  # Tokenize
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
  )

  # Generate
  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt

  return generated_text_answer

## computing precision, recall on training and validation data

In [25]:
text=np.array(train_df['question'])
pred_cat3=[]

for i,txt in enumerate(text):
  pred_category=inference(txt,model,tokenizer)
  pred_cat3.append(pred_category[0])

cat3=np.array(train_df['Cat3'])
precision, recall, f1, _ = precision_recall_fscore_support(cat3, pred_cat3, average='weighted')
accuracy = accuracy_score(cat3, pred_cat3)
print(precision, recall, f1, accuracy)

0.8426441902500093 0.8425827107790822 0.8328810245083068 0.8425827107790822


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [26]:
text=np.array(val_df['question'])
pred_cat3=[]

for i,txt in enumerate(text):
  pred_category=inference(txt,model,tokenizer)
  pred_cat3.append(pred_category[0])

cat3=np.array(val_df['Cat3'])
precision, recall, f1, _ = precision_recall_fscore_support(cat3, pred_cat3, average='weighted')
accuracy = accuracy_score(cat3, pred_cat3)
print(precision, recall, f1, accuracy)

0.7945041816601798 0.7918612408272181 0.7822668503286893 0.7918612408272181


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Prediction on unseen test data using predicted Cat1

In [27]:

text=np.array(test_df['question'])
pred_cat3=[]

for i,txt in enumerate(text):
  pred_category=inference(txt,model,tokenizer)
  pred_cat3.append(pred_category[0])

#append the predCat3 to testing data
test_df['pred_Cat3']=pd.Series(pred_cat3)
test_df['pred_Cat3']=test_df['pred_Cat3'].apply(lambda x: x.replace("_", " "))


In [28]:
cat3=np.array(test_df['Cat3'])
pred_cat3=np.array(test_df['pred_Cat3'])
precision, recall, f1, _ = precision_recall_fscore_support(cat3, pred_cat3, average='weighted')
accuracy = accuracy_score(cat3, pred_cat3)
print(precision, recall, f1, accuracy)

0.6215848017424541 0.646 0.6199687979831286 0.646


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [29]:
#saving the test data (Now having predCat1, predCat2 and predCat3)
test_df.drop(columns=['question']).to_csv("testing_data_pred_Cat_1_2_3.csv")